# Large-Scale Data Engineering for AI
Per a fer aquest projecte hem hagafat tres datasets relacionats amb música. Més concretament relacionats amb cançons que te Spotify. A continuació deixem una petita descripció de cadascun:
- **song-lyrics** --> conté unes 25000 cançons amb les seves corresponents lletres.
- **spotify-tracks-dataset** --> conté cançons d'Spotify de 125 gèneres diferents i altres atributs.
- **top-spotify-songs-by-country** --> conté les cançons més populars de 72 països. S'actualitza diàriament.

## The Data Engineering Pipeline
### The Landing Zone
Pel que fa a la landing zone, hem accedit a l'API de Kaggle per a obtenir els datasets. Un pas previ que s'ha de fer per a poder tenir accés és generar un token de Kaggle. Primer has d'entrar en el teu perfil i anar a la configuració, baixant es troba un apartat on hi diu "Create New Token" i d'allà es descarregarà un fitxer .json. Hem utilitzat la [guia](https://www.kaggle.com/docs/api#getting-started-installation-&-authentication) d'ús de l'API per aconseguir-ho. S'hauran de fer aquestes comandes:
```
pip install kaggle
mkdir ~/.kaggle
mv ~/Downloads/kaggle.json ~/.kaggle/kaggle.json
```

Un cop ja tenim accés a la API, hem fet una funció recol·lectora de dades per a baixar els tres datasets i guardar-los a l'adreça desitjada. Aquests datasets estan en format .csv i nosaltres hem preferit passar-los a format .parquet. A l'adreça del dataset de les lletres no només hi ha les lletres sinó que hi ha dos datasets més amb informació sobre els àlbums i més informació sobre les cançons. És per això que només ens hem de quedar amb el de lletres. En conclusió, ens baixem el dataset de l'API, el passem a .parquet, el guardem a la carpeta de "data/landing_zone". Al llarg d'aquesta execució, es van fent moltes comprovacions, en cas que algun moment falli es tingui proouta informació per trobar l'error.

In [16]:
# Imports and requirements
!pip install kaggle
!pip install pyspark

UnboundLocalError: cannot access local variable 'child' where it is not associated with a value

In [17]:
# Imports and requirements
import os
import datetime
import kaggle
import pandas as pd
import logging
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, when, regexp_replace, trim, lower, upper, to_date, year, month, dayofmonth, explode, split

In [18]:
# Base directory and Landing Zone directory structure
BASE_DIR = "./data"
LANDING_ZONE_DIR = os.path.join(BASE_DIR, "landing_zone")
os.makedirs(LANDING_ZONE_DIR, exist_ok=True)

kaggle.api.authenticate()

# List of datasets to ingest from Kaggle
datasets = [
        {
            "kaggle_id": "asaniczka/top-spotify-songs-in-73-countries-daily-updated",
            "dataset_name": "top-spotify-songs-by-country",
            "update": True
        },
        {
            "kaggle_id": "maharshipandya/-spotify-tracks-dataset",
            "dataset_name": "spotify-tracks-dataset",
            "update": False
        },
        {
            "kaggle_id": "terminate9298/songs-lyrics",
            "dataset_name": "songs-lyrics",
            "update": False
        }
        ]

In [19]:
# Logging function to timestamp each message
def log(message):
    print(f"[{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {message}")

def data_collector_kaggle(kaggle_dataset: dict) -> None:
    """
    Downloads a dataset from Kaggle and saves it to the landing zone.

    Parameters:
    kaggle_dataset (dict): A dictionary containing the Kaggle dataset information.
    """

    # Extract dataset information
    kaggle_id = kaggle_dataset["kaggle_id"]
    dataset_name = kaggle_dataset["dataset_name"]
    
    try:  
        log(f"Downloading dataset: {kaggle_id}")

        kaggle.api.dataset_download_files(
            kaggle_id,
            path=LANDING_ZONE_DIR,
            unzip=True
        )

        csv_found = False
        for filename in os.listdir(LANDING_ZONE_DIR):
            if filename in ['songs_details.csv', 'album_details.csv']:
                csv_path = os.path.join(LANDING_ZONE_DIR, filename)
                os.remove(csv_path)
                continue
            if filename.endswith(".csv"):
                csv_found = True
                csv_path = os.path.join(LANDING_ZONE_DIR, filename)

                # Read CSV with Pandas
                df = pd.read_csv(csv_path)

                # Write Parquet
                final_path = os.path.join(LANDING_ZONE_DIR, f"{dataset_name}.parquet")
                df.to_parquet(final_path, index=False)

                log(f"CSV '{filename}' converted to single Parquet file and saved as '{final_path}'.")

                os.remove(csv_path)
                        
        if not csv_found:
            log(f"No CSV file found in the downloaded dataset. Check the contents of the download.")

    except Exception as e:
        # Log the error
        log(f"Error downloading dataset '{kaggle_id}': {e}")
        return

    # Log the successful download
    log(f"Dataset '{dataset_name}' downloaded successfully.")

def download_and_store_datasets(update: bool = False) -> None:
    """
    Downloads and stores datasets from Kaggle into the landing zone.
    """
    log("Starting the creation of the Landing Zone using Kaggle API")

    for kaggle_dataset in datasets:
        if update and not kaggle_dataset["update"]:
            log(f"Skipping dataset '{kaggle_dataset['dataset_name']}' as update is set to False.")
            continue
        try:
            dataset_name = kaggle_dataset["dataset_name"]
            data_collector_kaggle(kaggle_dataset)
            log(f"Dataset '{dataset_name}' processed successfully.")
        except Exception as e:
            log(f"Error processing dataset '{dataset_name}': {e}")

    log("All datasets have been processed.")
    log("Landing Zone creation completed.")

download_and_store_datasets(update=False)

[2025-04-12 14:59:39] Starting the creation of the Landing Zone using Kaggle API
[2025-04-12 14:59:39] Downloading dataset: asaniczka/top-spotify-songs-in-73-countries-daily-updated
Dataset URL: https://www.kaggle.com/datasets/asaniczka/top-spotify-songs-in-73-countries-daily-updated
[2025-04-12 14:59:56] CSV 'universal_top_spotify_songs.csv' converted to single Parquet file and saved as './data/landing_zone/top-spotify-songs-by-country.parquet'.
[2025-04-12 14:59:56] Dataset 'top-spotify-songs-by-country' downloaded successfully.
[2025-04-12 14:59:56] Dataset 'top-spotify-songs-by-country' processed successfully.
[2025-04-12 14:59:56] Downloading dataset: maharshipandya/-spotify-tracks-dataset
Dataset URL: https://www.kaggle.com/datasets/maharshipandya/-spotify-tracks-dataset
[2025-04-12 14:59:58] CSV 'dataset.csv' converted to single Parquet file and saved as './data/landing_zone/spotify-tracks-dataset.parquet'.
[2025-04-12 14:59:58] Dataset 'spotify-tracks-dataset' downloaded succes

### The Formatted Zone

In [20]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

def create_spark_session(app_name="Spotify_ETL"):
    """
    Creates and returns a Spark session.
    
    Args:
        app_name (str): The name of the Spark application.
        
    Returns:
        SparkSession: A configured Spark session.
    """
    logger.info(f"Creating Spark session with app name: {app_name}")
    
    try:
        # Create a SparkSession with appropriate settings
        spark = (SparkSession.builder
                .appName(app_name)
                .config("spark.sql.execution.arrow.pyspark.enabled", "true")
                .config("spark.sql.shuffle.partitions", "10")
                .config("spark.driver.memory", "2g")
                .config("spark.executor.memory", "4g")
                .config("spark.default.parallelism", "4")
                .config("spark.sql.adaptive.enabled", "true")
                .getOrCreate())
        
        # Set log level to ERROR to reduce verbosity
        spark.sparkContext.setLogLevel("ERROR")
        
        logger.info("Spark session created successfully")
        return spark
    
    except Exception as e:
        logger.error(f"Error creating Spark session: {e}")
        raise

spark = create_spark_session()

2025-04-12 15:00:01,077 - __main__ - INFO - Creating Spark session with app name: Spotify_ETL
2025-04-12 15:00:01,100 - __main__ - INFO - Spark session created successfully


In [21]:
def process_spotify_tracks(spark, input_path, output_path):
    """
    Process the Spotify tracks dataset.
    
    Args:
        spark (SparkSession): The Spark session.
        input_path (str): The input file path.
        output_path (str): The output directory path.
    """
    logger.info(f"Processing Spotify tracks dataset from {input_path}")
    
    try:
        # Read the parquet file
        df = spark.read.parquet(input_path)
        
        # Print schema and count before processing
        logger.info("Original schema:")
        df.printSchema()
        count_before = df.count()
        logger.info(f"Count before processing: {count_before}")
        
        # Clean and transform the data
        processed_df = df.select(
            col("track_id").alias("track_id"),
            col("track_name").alias("track_name"),
            col("artists").alias("artist_name"),
            col("album_name"),
            col("popularity").cast("integer").alias("popularity"),
            col("duration_ms").cast("long").alias("duration_ms"),
            col("explicit").cast("boolean").alias("explicit"),
            col("danceability").cast("double").alias("danceability"),
            col("energy").cast("double").alias("energy"),
            col("key").cast("integer").alias("key"),
            col("loudness").cast("double").alias("loudness"),
            col("mode").cast("integer").alias("mode"),
            col("speechiness").cast("double").alias("speechiness"),
            col("acousticness").cast("double").alias("acousticness"),
            col("instrumentalness").cast("double").alias("instrumentalness"),
            col("liveness").cast("double").alias("liveness"),
            col("valence").cast("double").alias("valence"),
            col("tempo").cast("double").alias("tempo")
        )
        
        # Remove rows with null track_id or track_name
        # TRUSTED ZONE
        # processed_df = processed_df.filter(
        #     col("track_id").isNotNull() & 
        #     col("track_name").isNotNull()
        # )
        
        # Print schema and count after processing
        logger.info("Processed schema:")
        processed_df.printSchema()
        count_after = processed_df.count()
        logger.info(f"Count after processing: {count_after}")
        logger.info(f"Removed {count_before - count_after} rows during processing")
        
        # Write the processed data as Parquet
        processed_df.write.mode("overwrite").parquet(output_path)
        logger.info(f"Processed Spotify tracks data saved to {output_path}")
        
        return output_path
        
    except Exception as e:
        logger.error(f"Error processing Spotify tracks dataset: {e}")
        raise

def process_top_songs(spark, input_path, output_path):
    """
    Procesa el dataset de Spotify con el siguiente schema:
    
    root
     |-- spotify_id: string (nullable = true)
     |-- name: string (nullable = true)
     |-- artists: string (nullable = true)
     |-- daily_rank: string (nullable = true)
     |-- daily_movement: string (nullable = true)
     |-- weekly_movement: string (nullable = true)
     |-- country: string (nullable = true)
     |-- snapshot_date: string (nullable = true)
     |-- popularity: string (nullable = true)
     |-- is_explicit: string (nullable = true)
     |-- duration_ms: string (nullable = true)
     |-- album_name: string (nullable = true)
     |-- album_release_date: string (nullable = true)
     |-- danceability: string (nullable = true)
     |-- energy: string (nullable = true)
     |-- key: string (nullable = true)
     |-- loudness: string (nullable = true)
     |-- mode: string (nullable = true)
     |-- speechiness: string (nullable = true)
     |-- acousticness: double (nullable = true)
     |-- instrumentalness: double (nullable = true)
     |-- liveness: double (nullable = true)
     |-- valence: double (nullable = true)
     |-- tempo: double (nullable = true)
     |-- time_signature: double (nullable = true)
    
    Args:
        spark (SparkSession): La sesión de Spark.
        input_path (str): Ruta del archivo CSV de entrada.
        output_path (str): Ruta del directorio de salida en HDFS.
    """
    logger.info(f"Procesando datos de Spotify desde {input_path}")
    
    try:
        # Leer el archivo parquet
        df = spark.read.parquet(input_path)
        
        # Imprimir schema y contar filas antes del procesamiento
        logger.info("Schema original:")
        df.printSchema()
        count_before = df.count()
        logger.info(f"Filas antes del procesamiento: {count_before}")
        
        # Seleccionar y transformar las columnas según el nuevo schema
        processed_df = df.select(
            col("spotify_id").alias("spotify_id"),
            col("name").alias("track_name"),        
            col("artists").alias("artist_name"),
            col("daily_rank").cast("integer").alias("daily_rank"),
            col("daily_movement").alias("daily_movement"),
            col("weekly_movement").alias("weekly_movement"),
            col("country").alias("country"),
            col("snapshot_date").alias("snapshot_date"),
            col("popularity").cast("integer").alias("popularity"),
            col("is_explicit").alias("is_explicit"),
            col("duration_ms").cast("long").alias("duration_ms"),
            col("album_name").alias("album_name"),
            col("album_release_date").alias("album_release_date"),
            col("danceability").alias("danceability"),
            col("energy").alias("energy"),
            col("key").alias("key"),
            col("loudness").alias("loudness"),
            col("mode").alias("mode"),
            col("speechiness").alias("speechiness"),
            col("acousticness").cast("double").alias("acousticness"),
            col("instrumentalness").cast("double").alias("instrumentalness"),
            col("liveness").cast("double").alias("liveness"),
            col("valence").cast("double").alias("valence"),
            col("tempo").cast("double").alias("tempo"),
            col("time_signature").cast("double").alias("time_signature")
        )
        
        # Manejo de valores nulos para algunas columnas numéricas
        # TRUSTED ZONE
        # processed_df = processed_df.na.fill({
        #     "daily_rank": 0,
        #     "popularity": 0,
        #     "duration_ms": 0,
        #     "acousticness": 0.0,
        #     "instrumentalness": 0.0,
        #     "liveness": 0.0,
        #     "valence": 0.0,
        #     "tempo": 0.0,
        #     "time_signature": 0.0
        # })
        
        # Convertir snapshot_date y album_release_date a formato fecha (ajustar el patrón si es necesario)
        processed_df = processed_df.withColumn("snapshot_date", to_date(col("snapshot_date"), "yyyy-MM-dd"))
        processed_df = processed_df.withColumn("album_release_date", to_date(col("album_release_date"), "yyyy-MM-dd"))
        
        # Extraer año, mes y día a partir de snapshot_date para posibles análisis adicionales
        processed_df = processed_df.withColumn("snapshot_year", year(col("snapshot_date"))) \
                                   .withColumn("snapshot_month", month(col("snapshot_date"))) \
                                   .withColumn("snapshot_day", dayofmonth(col("snapshot_date")))
        
        # Limpiar el campo country: eliminar espacios extras y convertir a mayúsculas
        processed_df = processed_df.withColumn(
            "country", 
            upper(trim(regexp_replace(col("country"), "\\s+", " ")))
        )
        
        # Filtrar filas donde el nombre de la canción (track_name) no sea nulo
        # TRUSTED ZONE
        # processed_df = processed_df.filter(col("track_name").isNotNull())
        
        # Imprimir el schema y contar las filas después del procesamiento
        logger.info("Schema procesado:")
        processed_df.printSchema()
        count_after = processed_df.count()
        logger.info(f"Filas después del procesamiento: {count_after}")
        logger.info(f"Se removieron {count_before - count_after} filas durante el procesamiento")
        
        # Escribir el DataFrame procesado en formato Parquet en HDFS
        processed_df.write.mode("overwrite").parquet(output_path)
        logger.info(f"Datos procesados de Spotify guardados en {output_path}")
        
        return output_path
        
    except Exception as e:
        logger.error(f"Error al procesar los datos de Spotify: {e}")
        raise

def process_song_lyrics(spark, input_path, output_path):
    """
    Process the song lyrics dataset.
    
    Args:
        spark (SparkSession): The Spark session.
        input_path (str): The input file path.
        output_path (str): The output directory path.
    """
    logger.info(f"Processing song lyrics from {input_path}")
    
    try:
        # Read the CSV file with proper encoding
        df = spark.read.parquet(input_path)
        
        # Print schema and count before processing
        logger.info("Original schema:")
        df.printSchema()
        count_before = df.count()
        logger.info(f"Count before processing: {count_before}")
        
        # Clean and transform the data based on the actual columns in songs-lyrics.csv
        processed_df = df.select(
            col("Unnamed: 0").cast("integer").alias("song_id"),
            col("artist").alias("artist_name"),
            col("song_name").alias("song_name"),
            col("lyrics").alias("song_lyrics")
        )
        
        # Clean artist and track names
        processed_df = processed_df.withColumn("artist_name", trim(col("artist_name"))) \
                                 .withColumn("song_name", trim(col("song_name")))
        
        # Filter rows with valid song_id and track_name
        # TRUSTED ZONE
        # processed_df = processed_df.filter(col("song_id").isNotNull() & col("track_name").isNotNull())
        
        # Print schema and count after processing
        logger.info("Processed schema:")
        processed_df.printSchema()
        count_after = processed_df.count()
        logger.info(f"Count after processing: {count_after}")
        logger.info(f"Removed {count_before - count_after} rows during processing")
        
        # Write the processed data as Parquet
        processed_df.write.mode("overwrite").parquet(output_path)
        logger.info(f"Processed song lyrics data saved to {output_path}")
        
        return output_path
        
    except Exception as e:
        logger.error(f"Error processing song lyrics dataset: {e}")
        raise

In [ ]:
# Define input and output paths
input_paths = {
    'spotify_tracks': './data/landing_zone/spotify-tracks-dataset.parquet',
    'top_songs': './data/landing_zone/top-spotify-songs-by-country.parquet',
    'song_lyrics': './data/landing_zone/songs-lyrics.parquet'
}
output_paths = {
    'spotify_tracks': './data/formatted_zone/spotify-tracks-dataset',
    'top_songs': './data/formatted_zone/top-spotify-songs-by-country',
    'song_lyrics': './data/formatted_zone/songs-lyrics'
}

# Process datasets
process_spotify_tracks(spark, input_paths['spotify_tracks'], output_paths['spotify_tracks'])
process_top_songs(spark, input_paths['top_songs'], output_paths['top_songs'])
process_song_lyrics(spark, input_paths['song_lyrics'], output_paths['song_lyrics'])

### The Trusted Zone

### The Exploitation Zone

## The Data Analysis Pipelines